In [14]:
import math
import pandas as pd
import shutil
import numpy as np
from numpy import matrix
from decimal import *
import MDAnalysis as mda

/u/adlouet/conda-envs/tunnel/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Definition Functions:

In [15]:
def round(x):
    rounded=Decimal(x).quantize(Decimal('1e-3'))
    if rounded <100:
        space=(' '+str(rounded))
        return space
    else:
        return str(rounded)

In [16]:
def is_nan(x):
    return (x != x)

In [17]:
def calculate_pos(pos_A, pos_B, pos_C, length, angle, dihedral):
    
    #Calculates the position of an atom D based on 3 reference atoms A, B and C, with the following topology:
    #
    #                D
    #               /
    #              C
    #              |
    #              B
    #             /
    #            A
    #
    # with "length" being the distance between C and D
    #      "angle" being the angle between B, C and D
    #      "dihedral" being the dihedral angle between A, B, C and D
    #
    
    effective_angle=(float(angle)-90.0)*2*np.pi/360
    dihedral = dihedral*2*np.pi/360
    
    #Project A onto a plane with normal vector B-C and with C on the plane
    # A'=A+s(C-B)
    
    s = ((pos_C[0]-pos_A[0])*(pos_C[0]-pos_B[0])+(pos_C[1]-pos_A[1])*(pos_C[1]-pos_B[1])+(pos_C[2]-pos_A[2])*(pos_C[2]-pos_B[2]))/(((pos_C[0]-pos_B[0])**2)+((pos_C[1]-pos_B[1])**2)+((pos_C[2]-pos_B[2])**2))
 
    a = matrix(pos_A)
    b = matrix(pos_B)
    c = matrix(pos_C)
    
    pos_A_pro_mat = a + s*(c-b)
    
    pos_A_pro_mats=pos_A_pro_mat
    
    pos_A_pro_mat=pos_A_pro_mat.tolist()
    
    pos_A_pro = []

    for i in pos_A_pro_mat:
        for j in i:
            pos_A_pro.append(float(j))
            
    # Find point D' on BC plane, such that the angle between D'C and A'C is the dihedral angle
    
    # Substitutions:
    
    ux = (pos_A_pro[0]-pos_C[0])-((pos_C[0]-pos_B[0])*(pos_A_pro[2]-pos_C[2])/(pos_C[2]-pos_B[2]))
    uy = (pos_A_pro[1]-pos_C[1])-((pos_C[1]-pos_B[1])*(pos_A_pro[2]-pos_C[2])/(pos_C[2]-pos_B[2]))
    
    h = np.cos(effective_angle)*length
    n = h*np.linalg.norm(pos_A_pro_mats-c)
    
    wy0 = (n*np.cos(dihedral))**2+(((pos_C[0]-pos_B[0])*n*np.cos(dihedral))/(pos_C[2]-pos_B[2]))**2-(h*ux)**2
    wy1 = -2*n*np.cos(dihedral)*uy+(2*((pos_C[0]-pos_B[0])*n*np.cos(dihedral)*((pos_C[1]-pos_B[1])*ux-(pos_C[0]-pos_B[0])*uy))/((pos_C[2]-pos_B[2])**2))
    wy2 = uy**2+ux**2+(((pos_C[1]-pos_B[1])*ux-(pos_C[0]-pos_B[0])*uy)/(pos_C[2]-pos_B[2]))**2
    
    #x = d'_x - c_x 
    #y = d'_y - c_y
    #z = d'_z - c_z

    #y - two solutions
    y1 = -(wy1/(2*wy2)) + np.sqrt((wy1/(2*wy2))**2-(wy0/wy2)) 
    y2 = -(wy1/(2*wy2)) - np.sqrt((wy1/(2*wy2))**2-(wy0/wy2))
    #x - two solutions
    if ux == 0 and (n*np.cos(dihedral)-(uy*y1)) == 0:
        x1 = 0.0
    elif ux != 0:
        x1 = (n*np.cos(dihedral)-(uy*y1))/ux
    else:
        raise ValueError("Division by 0 (ux=0).")
    if ux == 0 and (n*np.cos(dihedral)-(uy*y2)) == 0:
        x2 = 0.0
    elif ux != 0:
        x2 = (n*np.cos(dihedral)-(uy*y2))/ux
    else:
        raise ValueError("Division by 0 (ux=0).")    
    #z - two solutions
    if ux == 0 and (pos_C[0]-pos_B[0]) == 0:
        z1 = -1/(pos_C[2]-pos_B[2])*((pos_C[1]-pos_B[1])*y1)
        z2 = -1/(pos_C[2]-pos_B[2])*((pos_C[1]-pos_B[1])*y2)
    elif ux != 0:
        z1 = -1/(pos_C[2]-pos_B[2])*(((pos_C[0]-pos_B[0])*n*np.cos(dihedral))/ux-(pos_C[0]-pos_B[0])*(uy/ux)*y1+(pos_C[1]-pos_B[1])*y1)
        z2 = -1/(pos_C[2]-pos_B[2])*(((pos_C[0]-pos_B[0])*n*np.cos(dihedral))/ux-(pos_C[0]-pos_B[0])*(uy/ux)*y2+(pos_C[1]-pos_B[1])*y2)
    else:
        raise ValueError("Division by 0 (ux=0).")
      
    #Point D' - two solutions
    
    pos_D_pro1 = np.array([x1+pos_C[0],y1+pos_C[1], z1+pos_C[2]])
    pos_D_pro2 = np.array([x2+pos_C[0],y2+pos_C[1], z2+pos_C[2]])

    #Point D -two solutions : 
    # D = D' + (C-B)/|C-B| * length * sin(effective_angle)
    return pos_D_pro1, pos_D_pro2
    #return pos_D1, pos_D2

In [18]:
def position_coordinates(pos):
    if len(pos)== 5:
        new_pos='   ' +pos
    elif len(pos)== 6:
        new_pos='  '+pos
    else :
        new_pos=' '+pos
        
    return new_pos

In [19]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','')

In [20]:
def change_atom_number(path):
    infile=glob.glob(path+'/pair*.gro')
    infile=str(infile)
    infile=listToStringWithoutBrackets(infile)
    infile = infile.replace("'", "")

    #print(infile)
    outfile=(path+'/out_lam_z.gro')
    #print(outfile)

    shutil.copyfile(infile, outfile)
    
    with open(outfile, 'r', encoding='utf-8') as file:
        data = file.readlines()

    
    with open(outfile, 'r') as fh:
        atom_og=(fh.readlines()[1])
        atom=int(atom_og)+1
             
    atoms=str(atom)+'\n'
    

    with open(outfile,"r") as f:
        newline=[]
        
        for word in f.readlines():        
            newline.append(word.replace(atom_og,atoms))
        
    with open(outfile,"w") as f:
        for line in newline:
            f.writelines(line)
    #print('done.. now to add H to newline')
    
    #return (outfile)

    length_H=0.96
    angle_H=107
    dihedral_H=-45.58
    u=[]

    OJ_word="4SP2     OG"
    P_word="4SP2      P"
    OG_word="4SP2     OJ"
    
    with open(outfile, 'r') as fh:
        for line in fh:
            if OJ_word in line:
                u.append(line)
            if P_word in line:
                u.append(line)
            if OG_word in line:
                u.append(line)
                
    df=pd.DataFrame(u)
    cols = np.arange(df[0].str.split(expand = True).shape[1])
    df[cols] = df[0].str.split(expand = True)
    
    pos_A_whole=df[df[1].str.contains("OG") ]
    pos_A_df=(pos_A_whole[3]).tolist(),(pos_A_whole[4]).tolist(),(pos_A_whole[5]).tolist()

    pos_A = []
    for i in pos_A_df:
        for j in i:
            pos_A.append(float(j))
            
    pos_B_whole=df[df[1].str.contains("P") ]
    pos_B_df=(pos_B_whole[3]).tolist(),(pos_B_whole[4]).tolist(),(pos_B_whole[5]).tolist()

    pos_B = []
    for i in pos_B_df:
        for j in i:
            pos_B.append(float(j)) 
            
    pos_C_whole=df[df[1].str.contains("OJ") ]
    pos_C_df=(pos_C_whole[3]).tolist(),(pos_C_whole[4]).tolist(),(pos_C_whole[5]).tolist()

    pos_C = []
    for i in pos_C_df:
        for j in i:
            pos_C.append(float(j))    
            
    #p=calculate_pos(pos_A, pos_B, pos_C, length_H, angle_H, dihedral_H)
    try:
        p=calculate_pos(pos_A, pos_B, pos_C, length_H, angle_H, dihedral_H)
    except ZeroDivisionError:
        p=(([0.0000000 , 0.0000000, 0.0000000]),([0.0000000, 0.0000000, 0.0000000]))
        print(infile,"broke")
        #pass
    
    pos_p = []
                ##(was previously for i in y but doesnt work anymore)
    for i in p:
        for j in i:
            pos_p.append(float(j))
    state=is_nan(pos_p[0]) 
                #print(state)

    if state == True:
        print( word + " doesn't work")

    if state == False:
        pass

        pos_p_x=round(pos_p[0])
        pos_p_y=round(pos_p[1])
        pos_p_z=round(pos_p[2])    
        
    #new_line_in="    4SP2     ZH   52" +position_coordinates(pos_p_x) + position_coordinates(pos_p_y) + position_coordinates(pos_p_z)
    new_line_in="    4SP2     ZH"+ "     " +position_coordinates(pos_p_x) + position_coordinates(pos_p_y) + position_coordinates(pos_p_z)

    call='SP2    O2P'
    inputfile = open(outfile, 'r').readlines()
    write_file = open(outfile,'w')

    for line in inputfile:
        write_file.write(line)
        if call in line:
            new_line =new_line_in          
            write_file.write(new_line + "\n") 
    write_file.close()               
